# Desafio 7 - Classiﬁcação de tweets utilizando o BERT

## Objetivo

Utilizar um modelo pré-treinado de PLN (BERT) para classiﬁcar o
sentimento de tweets em positivo ou negativo.

### Preparação dos Dados:

- Utilize um conjunto de dados de avaliações de tweets rotulados com sentimento (positivo/negativo).
- Divida o conjunto de dados em conjuntos de treinamento e teste.

### Pré-processamento dos Dados:

- Limpeza e tokenização dos textos dos tweets.
- Codiﬁcação dos tokens utilizando o vocabulário do modelo BERT.
- Adição de tokens especiais para separar frases e indicar o início e ﬁm do texto.

### Fine-tuning do Modelo BERT:

- Carregue o modelo pré-treinado BERT.
- Adicione camadas adicionais para a classiﬁcação de sentimento.
- Deﬁna a função de perda e o otimizador.
- Treine o modelo utilizando o conjunto de treinamento.

### Avaliação do Modelo:

- Avalie o modelo utilizando o conjunto de teste.
- Calcule a precisão, recall, e outras métricas de avaliação.

### Aplicação do Modelo:

- Teste o modelo com tweets não vistos antes para veriﬁcar sua eﬁcácia na classiﬁcação de sentimentos.

## Importando os pacotes necessarios

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from torch import nn, optim, tensor, no_grad, max
from matplotlib import pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np

## Carregando o dataset e salvando em uma variável

In [ ]:
path = "./datasets/raw/Twitter_Data.csv"
df_twitter_raw = pd.read_csv(path, sep = ",")

## Informações sobre os dados contidos no dataset

### Valores aleatórios

In [ ]:
df_twitter_raw.sample(5)

### Informaçõees detalhadas

In [ ]:
df_twitter_raw.info()

### Quantidade total de linhas do dataset

In [ ]:
print(f"A quantidade total de linhas é: {df_twitter_raw.shape[0]}")

### EDA e Tratamento dos dados

### Tratamento dos dados

#### Copiando o dataset em nova variável para realizar os tratamentos de forma segura

In [ ]:
df_twitter_processed = df_twitter_raw.copy(deep = True)
df_twitter_processed.sample(5)

#### Renomeando as colunas

##### Verificando quais são as colunas contidas no dataset

In [ ]:
df_twitter_processed.columns

##### Realizando a renomeação

In [ ]:
novas_colunas_nome = ["texto", "emocao"]
df_twitter_processed.columns = novas_colunas_nome
df_twitter_processed.sample(5)

#### Alterando os valores da coluna "*emocao*"

##### Verificando os valores atuais

In [ ]:
df_twitter_processed["emocao"].unique()

Valores não-númericos/nulos foram achados, vamos tratar isso abaixo

##### Verificando os valores não-númericos/nulos na coluna "*emocao*"

In [ ]:
df_twitter_processed["emocao"].isna().sum()

##### Verificando os valores não-númericos/nulos por todo o dataset

In [ ]:
df_twitter_processed.isna().sum()

Como há somente 4 linhas faltantes da coluna "*texto*", será optado por excluir completamente as linhas

A quantidade de linhas excluidas não irá afetar muito a quantidades de dados do dataset

##### Excluindo as linhas onde há valores não-numéricos/nulos na coluna "*texto*"

In [ ]:
df_twitter_processed = df_twitter_processed[~df_twitter_processed["texto"].isna()]

##### Nova contagem de valores não-numéricos/nulos

In [ ]:
df_twitter_processed.isna().sum()

As linhas onde a coluna "*emocao*" possui valores nãp-numéricos/nulos não coincidio com a coluna "*texto*".

No entanto ainda será optado por excluir as linhas em questão, já que sua quantidade é muito pequena em comparação com o total do dataset

##### Excluindo as linhas onde há valores não-numéricos/nulos na coluna "*emocao*"

In [ ]:
df_twitter_processed = df_twitter_processed[~df_twitter_processed["emocao"].isna()]

##### Nova contagem de valores não-numéricos/nulos

In [ ]:
df_twitter_processed.isna().sum()

Agora vamos alterar os valores da coluna "*emocao*"

##### Realizando a alteração dos valores

In [ ]:
dict_emocao_novos_valores = {
    -1 : 0, # NEGATIVO
    0 : 1,  # NEUTRO
    1 : 2   # POSITIVO
}

df_twitter_processed["emocao"] = df_twitter_processed["emocao"].map(dict_emocao_novos_valores)
df_twitter_processed["emocao"].unique()

#### Alterando os tipos de dados contidos no dataset

Coluna *emocao*:
- Atual -> float64
- Novo -> uint8

Coluna *texto*:
- Atual -> Object
- Novo -> String

In [ ]:
df_twitter_processed["emocao"] = df_twitter_processed["emocao"].astype("uint8")
df_twitter_processed["texto"] = df_twitter_processed["texto"].astype("string")

##### Verificando novos tipos

In [ ]:
df_twitter_processed.info()

#### Salvando dataset atual em formato .pkl

In [ ]:
path = "./datasets/processed/twitter_data.pkl"
df_twitter_processed.to_pickle(path)

### EDA

#### Quantidades unitárias de cada valor único da coluna "*emocao*"

In [ ]:
df_twitter_processed["emocao"].value_counts()

Podemos notar que:
- A maioria dos tweets é positivo, com o valor: 72249
- A quantidade de tweets neutros é de: 55211
- Os tweets negativos são a minoria, com o valor de: 35509

#### De forma gráfica

In [ ]:
imagem = plt.figure(figsize=(12, 8))
plt.hist(x=df_twitter_processed["emocao"])
plt.show()

#### Desvio Padrão e Média Aritmética

In [ ]:
print("O desvio padrão é de: ", df_twitter_processed["emocao"].std(),
    "\nA média aritmética é de: ", df_twitter_processed["emocao"].mean())

#### Quantidade total de valores

In [ ]:
print(f"{df_twitter_processed['emocao'].shape[0]} linhas")

#### Breve descrição

In [ ]:
df_twitter_processed.describe()

## Treinando o modelo BERT

### Separando o dataset em dados de entrada (X) e target (y)

In [ ]:
X = df_twitter_processed.drop(columns = ["emocao"], axis = 1)
y = df_twitter_processed["emocao"]

### Realizando a separação de dados de teste e dados de treino

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

### Gerando Tokens através do texto

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

#### Função para gerar e retornar os tokens através dos dados já separados

In [ ]:
def tokenizar_texto(array_texto:pd.Series) -> BertTokenizer:
    return tokenizer\
        .batch_encode_plus(
            array_texto.tolist(),
            add_special_tokens = True,
            max_length = 512,
            return_tensors = "pt",
            padding = True,
            truncation = True,
        )


#### Gerando os tokens através de x_train e x_test

In [ ]:
x_train = tokenizar_texto(x_train.iloc[:,0])
x_test = tokenizar_texto(x_test.iloc[:,0])

#### Convertendo as listas para Tensores

##### Treino

In [ ]:
x_train_sequencia = tensor(x_train["input_ids"])
x_train_mascara = tensor(x_train["attention_mask"])
y_train_tensor = tensor(y_train.tolist())

##### Teste

In [ ]:
x_test_sequencia = tensor(x_test["input_ids"])
x_test_mascara = tensor(x_test["attention_mask"])
y_test_tensor = tensor(y_test.tolist())

### Dataloader (Carregar os dados)

#### Tamanho dos dados

In [ ]:
#define a batch size
tamanho_batch = 32

#### Treino

In [ ]:
# wrap tensors
dados_treino = TensorDataset(
    x_train_sequencia, 
    x_train_mascara,
    y_train_tensor
)

# sampler for sampling the data during training
train_sampler = RandomSampler(dados_treino)

# dataLoader for train set
train_dataloader = DataLoader(
    dados_treino,
    sampler = train_sampler,
    batch_size = tamanho_batch
)

#### Teste

In [ ]:
# wrap tensors
dados_teste = TensorDataset(
    x_test_sequencia,
    x_test_mascara,
    y_test_tensor
)

# sampler for sampling the data during training
test_sampler = SequentialSampler(dados_teste)

# dataLoader for validation set
test_dataloader = DataLoader(
    dados_teste,
    sampler = test_sampler,
    batch_size = tamanho_batch
)

### Fine-tuning do modelo BERT

#### Carregando modelo

In [ ]:
modelo = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

#### Adicionando camadas

In [ ]:
modelo.classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)

#### Função de Perda

In [ ]:
criterio_perda = nn.CrossEntropyLoss()

#### Otimizador

In [ ]:
otimizador = optim.Adam(modelo.parameters(), lr = 1e-5)

#### Quantidade de Epocas

In [ ]:
numero_epocas = 5

#### Treinando o modelo

In [ ]:
for epoch in range(numero_epocas):
    modelo.train()
    
    for passo, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = tuple(t for t in batch)

        # Zerar gradientes
        criterio_perda.zero_grad()

        # Forward pass
        outputs = modelo(
            b_input_ids,
            attention_mask = b_input_mask,
            labels = b_labels
        )
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        loss.backward()
        otimizador.step()

        if passo % 100 == 0:
            print(f"==================================================\nÉpoca: {epoch}\nPasso: {passo}\nPerda: {loss.item()}")


#### Avaliando o modelo

In [ ]:
predicoes = []
valores_reais = []
modelo.eval()

for idx, batch in enumerate(test_dataloader):
    b_input_ids, b_input_mask, b_labels = tuple(t for t in batch)
    
    with no_grad():
        outputs = modelo(
            b_input_ids,
            attention_mask = b_input_mask
        )
    
    logits = outputs.logits
    _, resultados_preditos = max(logits, 1)
    
    predicoes.extend(resultados_preditos.cpu().numpy())
    valores_reais.extend(b_labels.cpu().numpy())

acuracia = accuracy_score(valores_reais, predicoes)
precisao = precision_score(valores_reais, predicoes)
recall = recall_score(valores_reais, predicoes)

print("Acurácia:", acuracia)
print("Precisão:", precisao)
print("Recall:", recall)